<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Gestión en Logística y Cadena de Suministro</strong></h1>
  <h3><strong>Ejercicio 1</strong></h3>
</div>

Un fabricante de juguetes está planeando producir $n$ nuevos modelos de juguetes en sus $m$ plantas productivas, debido a que con los modelos que se fabrican actualmente no se está utilizando toda la capacidad de las líneas de ensamblaje de sus plantas. Suponga que $b_i$ denota la capacidad productiva disponible en la planta $i$ para la fabricación de los nuevos modelos, expresada en horas. Adicionalmente, sean $f_j$ el costo fijo en el que incurre cada una de las plantas si decide fabricar el modelo $j$, $t_{ij}$ la tasa de producción (en unidades por hora) del modelo de juguete $j$ en la planta $i$, $p_j$ el precio de venta del modelo de juguete $j$, y $c_{ij}$ el costo variable de producción del modelo de juguete $j$ en la planta $i$.

Se requiere determinar lo siguiente:

- Modelos nuevos de juguetes a fabricar.
- Cantidad a producir de cada modelo nuevo de juguete en cada una de las plantas.

In [12]:
from ortools.linear_solver import pywraplp

In [13]:
# Crear solver
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjunto de Índices**

-  $n$: Número de nuevos modelos de juguetes a producir.
-  $m$: Número de plantas productivas disponibles.
-  $b_i$: Capacidad productiva disponible en la planta $i$, expresada en horas.
-  $f_j$: Costo fijo en el que incurre cada planta si decide fabricar el modelo $j$.
-  $t_{ij}$: Tasa de producción (en unidades por hora) del modelo de juguete $j$ en la planta $i$.
-  $p_j$: Precio de venta del modelo de juguete $j$.
-  $c_{ij}$: Costo variable de producción del modelo de juguete $j$ en la planta $i$.

### **Parámetros**

In [14]:
n = 5
m = 4
b = [1737, 2646, 2690, 1253]
f = [42000, 100000, 35000, 31000, 23000]
t = [
    [165, 163, 188, 145, 170],
    [91, 83, 146, 190, 191],
    [151, 175, 163, 147, 77],
    [84, 188, 122, 180, 107]
]
p = [56, 58, 45, 40, 38]
c = [
    [15, 17, 10, 7, 4],
    [10, 19, 7,  6, 4],
    [13, 15, 8,  7, 7],
    [11, 15, 10, 8, 7]
]

### **Variables de decisión**

$$
\text{Para}\quad i=1,...,m; \quad j=1,...,n: 
$$

$$
x_{ij} = \text{Unidades a producir del modelo $j$ en la planta $i$}

In [15]:
x = {} # x[i,j] = cantidad de producto i que se produce en la planta j
for i in range(m): # plantas
    for j in range(n): # productos
        x[(i, j)] = solver.IntVar(0, solver.infinity(), f'x[{i},{j}]') # intvar porque es la cantidad de productos


$$y_{ij} = \begin{cases} 
1, & \text{si se produce el modelo j en la planta i} \\
0, & \text{en otro caso}
\end{cases} 
$$

In [16]:
y = {} # y[i,j] = 1 si la planta j produce el producto i, 0 en otro caso
for i in range(m): # plantas
    for j in range(n): # productos
        y[(i, j)] = solver.BoolVar(f'y[{i},{j}]')

### **Función objetivo**

Objetivo: Maximizar el beneficio = ingresos - costos

$$
\max \sum_{j=1}^{n} p_j \left( \sum_{i=1}^{m} x_{ij} \right) - \left( \sum_{j=1}^{n} f_j \left( \sum_{i=1}^{m} y_{ij} \right) + \sum_{j=1}^{n} \sum_{i=1}^{m} c_j \cdot x_{ij} \right)
$$

In [17]:
beneficio = solver.Objective()

beneficio = (
    sum(p[j] * x[(i, j)] for i in range(m) for j in range(n)) # Ingresos
    - (sum(f[j] * y[(i, j)] for i in range(m) for j in range(n)) # Costos fijos
    + sum(c[i][j] * x[(i, j)] for i in range(m) for j in range(n))) # Costos variables
)


solver.Maximize(beneficio)

### **Restricciones**

Tenemos que sumar sobre los modelos. Me voy a gastar las horas que hago fabricando juguetes. 

$$
\begin{aligned}
\sum_{i=1}^{n} \frac{1}{t_{ij}} x_{ij} &\leq b_i \\
x_{ij} &\in \mathbb{Z}_+ \\
y_{ij} &\in \{0,1\} 
\end{aligned}
$$

Es $\frac{1}{t_{ij}} x_{ij}$ porque las unidades estan por hora

nos falta la regla!

$$
x \leq M \delta
$$

$$
x_{ij} \leq b_i \cdot t_{ij} \cdot y_{ij} 
$$

El modelo j en la planta i tiene que valer

Entonces las restricciones nos quedan de la siguiente manera:

$$
\begin{aligned}
\sum_{i=1}^{n} \frac{1}{t_{ij}} x_{ij} &\leq b_i \\
x_{ij} &\leq b_i \cdot t_{ij} \cdot y_{ij} \\
x_{ij} &\in \mathbb{Z}_+ \\
y_{ij} &\in \{0,1\} 
\end{aligned}
$$

$$
\begin{aligned}
\sum_{i=1}^{n} \frac{1}{t_{ij}} x_{ij} &\leq b_i \\
\end{aligned}
$$

In [18]:
for i in range(m):
    solver.Add(solver.Sum(x[(i, j)] / t[i][j] for j in range(n)) <= b[i])

$$
\begin{aligned}
x_{ij} &\leq b_i \cdot t_{ij} \cdot y_{ij} \\
\end{aligned}
$$

In [19]:
for j in range(n):
    for j in range(n):
        solver.Add(x[(i, j)] <= b[i] * t[i][j] * y[(i, j)])

Las otras 2 ya están cuando declaramos las variables.

### **Resolver**

In [20]:
solver.Solve()

0

In [21]:
print("Beneficio máximo =", solver.Objective().Value())

Beneficio máximo = 59205431.0


In [ ]:
for i in range(m):
    for j in range(n):
        if x[(i, j)].solution_value() > 0:
            print(f"Planta {i}, Modelo {j}: {x[(i, j)].solution_value()}")

Cantidades a producir por planta y modelo:
Planta 0, Modelo 0: 286605.0
Planta 1, Modelo 4: 505386.0
Planta 2, Modelo 1: 470750.0
Planta 3, Modelo 1: 235564.0
